# CICY 3-folds: ML analysis

Author: [Harold Erbin](mailto:erbin@mit.edu)

This is the notebook for the analysis done in [arxiv:2007.13379](https://arxiv.org/abs/2007.13379) and [arxiv:2007.15706](https://arxiv.org/abs/2007.15706).

Data is automatically downloaded in the current folder. It has been converted to HDF from the original data (see [cicy3f.README](http://www.lpthe.jussieu.fr/~erbin/files/data/cicy3f.README) and [cicy3o.README](http://www.lpthe.jussieu.fr/~erbin/files/data/cicy3o.README) for details on the original data) and includes engineered features (this the papers for details).

The code relies on the package `mltools` which I created for my own use: it is automatically downloaded and can be found on Github. I don't guarantee forward compatibility.

In [ ]:
import sys
import os
import json

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import keras
from tensorflow.keras import layers
from sklearn import preprocessing, model_selection

In [ ]:
if not os.path.exists('mltools.egg'):
    !wget http://www.lpthe.jussieu.fr/~erbin/files/data/mltools.egg

In [ ]:
eggpath = os.path.realpath('./mltools.egg')
if eggpath not in sys.path:
    sys.path.insert(0, eggpath)

import mltools
from mltools import CategoricalFeatures, DataStructure, DataExploration
from mltools import (LinearRegression, SVM, DecistionTree, RandomForest,
                     NeuralNet, RatioSample)
from mltools import Logger, Predictions
from mltools.data import datatools as dt

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

## Constants

In [ ]:
# true = keep outliers
OUTLIERS = ('all', 'test', None)[1]
SCALEMAT = (True, False)[1]

EDA = (True, False)[1]

LC = (True, False)[1]
FIT = (True, False)[0]

LINEAR = (True, False)[0]
NONLINEAR = (True, False)[0]

CONVNET = (True, False)[1]
INCNET = (True, False)[1]
ADVNET = (True, False)[1]

training_ratio = 0.8
samples = RatioSample([training_ratio, 0.1, 0.9 - training_ratio])

integers_fn = {"h11": np.round, "h21": np.round}
lin_integers_fn = {"h11": np.floor, "h21": np.round}

metrics = {"h11": "accuracy", "h21": "accuracy"}

In [ ]:
%%capture
RESULTPATH = "./results"
DATAPATH = os.path.realpath(".")

DATAFILE_FAV = 'cicy3f.h5'
DATAFILE_ORIG = 'cicy3o.h5'
DATAFILE = DATAFILE_ORIG

if not os.path.exists(DATAFILE_FAV):
    !wget http://www.lpthe.jussieu.fr/~erbin/files/data/cicy3f.h5
if not os.path.exists(DATAFILE_ORIG):
    !wget http://www.lpthe.jussieu.fr/~erbin/files/data/cicy3o.h5

In [ ]:
np.random.seed(42)

pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 20)
pd.set_option('display.float_format', "{:.3f}".format)

## Helpers

In [ ]:
def load_data(datapath, filename):

    return pd.read_hdf(os.path.join(datapath, filename))


def filter_outliers(df, h11_max=18, h21_max=70):

    h11_cutoff = df['h11'] <= h11_max
    h21_cutoff = df['h21'] <= h21_max
    notprod = df['isprod'] == 0

    return df[notprod & h11_cutoff & h21_cutoff]
    # return df[notprod]


def outliers_name(filename):
    if OUTLIERS is True:
        return logger.inserttofilename(filename, "_outliers")
    else:
        return filename


def print_results(pred, name=""):
    if name != "":
        name = " (%s)" % name

    print()
    print("# Model: {}{}".format(pred.model.model_name, name))

    try:
        h11_res = pred.feature_metric("h11", mode="text")
        print("h11: {}".format(h11_res))
    except KeyError:
        pass

    try:
        h21_res = pred.feature_metric("h21", mode="text")
        print("h21: {}".format(h21_res))
    except KeyError:
        pass
    
    print()


def learning_curve_plot(acc, filename=None, logtime=None):

    labels = list(list(list(acc.values())[0].values())[0].keys())

    ratios = []

    scores_train = {k: [] for k in labels}
    scores_test = {k: [] for k in labels}

    for x, sc in acc.items():
        ratios.append(x)
        for k, v in sc['train'].items():
            scores_train[k].append(v)
        for k, v in sc['test'].items():
            scores_test[k].append(v)

    scores_train = {k: np.array(v) for k, v in scores_train.items()}
    scores_test = {k: np.array(v) for k, v in scores_test.items()}

    fig, ax = plt.subplots()

    styles = logger.styles
    linestyle = ['solid', 'dashed']

    for i, k in enumerate(labels):
        if len(scores_train[k].shape) > 1:
            train_values = scores_train[k][:, 0]
            train_stat = scores_train[k][:, 1]
            test_values = scores_test[k][:, 0]
            test_stat = scores_test[k][:, 1]
        else:
            train_values = scores_train[k]
            train_stat = None
            test_values = scores_test[k]
            test_stat = None

        ax.plot(ratios, train_values, linestyle=linestyle[i], marker='.',
                color=styles["color:train"],
                label="{} ({})".format(k, styles["label:train"]))
        ax.plot(ratios, test_values, linestyle=linestyle[i], marker='.',
                color=styles["color:val"],
                label="{} ({}.)".format(k, styles["label:val"][:3]))
        
        if train_stat is not None:
            ax.fill_between(ratios, train_values - train_stat,
                    train_values + train_stat,
                    alpha=0.3, color=styles["color:train"])
            ax.fill_between(ratios, test_values - test_stat,
                    test_values + test_stat,
                    alpha=0.3, color=styles["color:val"])

    ax.legend()

    ax.set_xlabel("training ratio")
    ax.set_ylabel("accuracy")

    ax.set_ylim(ymin=0, ymax=1)

    if filename is not None:
        logger.save_fig(fig, filename=filename, logtime=logtime)
    
    plt.close(fig)
    
    return fig


def learning_curve(model_fn, integers_fn, train_params=None, ratios=None,
                   filename=None, logtime=True):
    
    if ratios is None:
        ratios = np.arange(0.1, 1.0, 0.1)

    metrics = {"h11": "accuracy", "h21": "accuracy"}

    lc_acc = {}

    for x in ratios:
        x = np.round(x, 2)

        print("ratio = ", x)

        trainval = cydata_all.sample(frac=x)
        test = cydata_all.loc[~cydata_all.index.isin(trainval.index)].sample(frac=1)

        if OUTLIERS == 'test':
            trainval = filter_outliers(trainval, h11_max=18, h21_max=70)

        model = model_fn()

        if isinstance(model, NeuralNet):

            val = trainval.sample(frac=0.1)
            train = trainval.loc[~trainval.index.isin(val.index)].sample(frac=1)

            model.fit(train, val_data=[val, val], train_params=train_params)
        else:
            model.fit(trainval)

        lc_pred_test = Predictions(test, model=model, metrics=metrics,
                                   integers_fn=integers_fn, logger=logger)

        lc_pred_train = Predictions(trainval, model=model, metrics=metrics,
                                    integers_fn=integers_fn, logger=logger)

        lc_acc[x] = {}
        lc_acc[x]["train"] = {f: lc_pred_train.feature_metric(f)
                              for f in model.outputs.features}
        lc_acc[x]["test"] = {f: lc_pred_test.feature_metric(f)
                             for f in model.outputs.features}

        if filename is not None:
            logger.save_json(lc_acc, filename=filename + '.json',
                             logtime=logtime)
            pdfname = filename + '.pdf'
        else:
            pdfname = None

        fig = learning_curve_plot(lc_acc, pdfname, logtime)
    
    print()
    
    return fig, lc_acc

def train_predict(inputs, outputs, Model, model_params, int_fn, n=1,
                  name="", filename=None):
    model = Model(inputs=inputs, outputs=outputs, n=n,
                  model_params=model_params)
    model.fit(cydata['train'])

    pred = build_pred(model, int_fn)

    print_results(pred, name)

    if filename is not None:
        pred.summary(filename=filename, density=False, logtime=True)

    return pred

## Data and logger

In [ ]:
logger = Logger(RESULTPATH, logtime="folder")

In [ ]:
cydata_all = load_data(DATAPATH, DATAFILE)
cydata_all = dt.pad_data(cydata_all)

if OUTLIERS is None:
    cydata_all = filter_outliers(cydata_all)

if SCALEMAT is True:
    cydata_all['matrix'] = cydata_all['matrix'].apply(lambda x: x / 5)

cydata = samples(cydata_all, shuffle=True)

if OUTLIERS == 'test':
    cydata['train'] = filter_outliers(cydata['train'])
    cydata['val'] = filter_outliers(cydata['val'])

def build_pred(model, int_fn):
    return Predictions(cydata['test'], model=model, metrics=metrics,
                       integers_fn=int_fn, logger=logger)

In [ ]:
outputs_list = ['h11', 'h21']
outputs = DataStructure(outputs_list, infer=cydata_all)
h11_outputs = DataStructure(['h11'], infer=cydata_all)
h21_outputs = DataStructure(['h21'], infer=cydata_all)
outputs = h21_outputs

scalar_inputs = ['num_cp']
aux_scalar_inputs = ['num_eqs',
                     'min_dim_cp', 'max_dim_cp', 'mean_dim_cp', 'median_dim_cp',
                     # 'num_cp_1', 'num_cp_2', 'num_cp_neq1',
                     'num_over', 'num_ex',
                     'max_deg_eqs', 'mean_deg_eqs', 'median_deg_eqs',
                     'rank_matrix', 'norm_matrix']
all_scalar_inputs = scalar_inputs + aux_scalar_inputs

vector_inputs = ['dim_h0_amb', 'dim_cp']
aux_vector_inputs = ['num_deg_eqs']
# 'num_dim_cp', 'deg_eqs'
all_vector_inputs = vector_inputs + aux_vector_inputs

matrix_inputs = ['matrix']

all_inputs = all_scalar_inputs + all_vector_inputs + matrix_inputs

inputs_tab = {"num_cp": scalar_inputs,
              "scalar": all_scalar_inputs,
              "dim_cp_h0": vector_inputs,
              "vector": all_vector_inputs,
              "num_dim_cp_h0": scalar_inputs + vector_inputs,
              "scalar+vector": all_scalar_inputs + all_vector_inputs,
              "matrix": matrix_inputs,
              "num_cp": scalar_inputs + matrix_inputs,
              "matrix+scalar": all_scalar_inputs + matrix_inputs,
              "matrix+dim_cp_h0": vector_inputs + matrix_inputs,
              "matrix+vector": all_vector_inputs + matrix_inputs,
              "all": all_inputs
              }

## Data exploration

In [ ]:
if EDA is True:
    
    de = DataExploration(logger=logger)
    de.summary(cydata_all, filename="eda")
    de.summary_io(cydata_all, inputs=all_scalar_inputs, outputs=outputs_list,
                filename="eda_io_scalar")
    de.importances(cydata_all, outputs=outputs_list,
                inputs=all_scalar_inputs + all_vector_inputs + matrix_inputs,
                label_rot=90, ymax=0.3, sum_tensor=False,
                filename="eda_importances")
    de.importances(cydata_all, outputs=outputs_list,
                inputs=all_scalar_inputs + all_vector_inputs + matrix_inputs,
                sum_tensor=True, filename="eda_importances_sum")
    de.importances(cydata_all, outputs=outputs_list,
                inputs=all_scalar_inputs, filename="eda_importances_scalars")
    de.importances(cydata_all, outputs=outputs_list,
                inputs=all_vector_inputs, filename="eda_importances_vectors")
    de.importances(cydata_all, outputs=outputs_list,
                inputs=matrix_inputs, ymax=0.15,
                filename="eda_importances_matrix")

## Linear regression

### Input: matrix

In [ ]:
inputs_list = ['matrix']
inputs = DataStructure(inputs_list)
name = 'matrix'

model_params = {"l1": 0.0002, "max_iter": 5000, "fit_intercept": False}

if LINEAR is True and FIT is True:
    pred = train_predict(inputs, outputs, LinearRegression, model_params,
                         int_fn=lin_integers_fn, name=name,
                         filename=outliers_name("linreg_summary_" + name))

if LINEAR is True and LC is True:
    model_fn = lambda: LinearRegression(inputs, outputs,
                                        model_params=model_params, n=1)

    acc, fig = learning_curve(model_fn, lin_integers_fn,
                              filename=outliers_name("linreg_learning_curve_" + name))

    display(fig)
    # print(json.dumps(acc, indent=2))
    print(acc)


### Input: num_cp

The next cell shows that it's possible to use np.round also for linear regression, but it's very specific and needs some fine tuning.

In [ ]:
inputs_list = ['num_cp']
inputs = DataStructure(inputs_list, infer=cydata_all)
name = "num_cp"

model_params = {"l1": 5, "l2": 0.01, "max_iter": 5000, "fit_intercept": False}

if LINEAR is True:
    pred = train_predict(inputs, outputs, LinearRegression, model_params,
                         int_fn=lin_integers_fn, name=name, filename=None)
    print("h11 = %.2f + %.3f #CP" % (pred.model.model.intercept_, pred.model.model.coef_))


In [ ]:
model_params = {"l1": 1, "max_iter": 5000, "fit_intercept": False}

if LINEAR is True and FIT is True:
    pred = train_predict(inputs, outputs, LinearRegression, model_params,
                         int_fn=lin_integers_fn, name=name,
                         filename=outliers_name("linreg_summary_" + name))

if LINEAR is True and LC is True:
    model_fn = lambda: LinearRegression(inputs, outputs,
                                        model_params=model_params, n=1)

    acc, fig = learning_curve(model_fn, lin_integers_fn,
                              filename=outliers_name("linreg_learning_curve_" + name))

    display(fig)
    print(acc)


### Input: vectors

In [ ]:
inputs_list = ['dim_cp', 'dim_h0_amb']
inputs = DataStructure(inputs_list, infer=cydata_all)
name = "vectors"

model_params = {"l1": 0.01, "max_iter": 5000}

if LINEAR is True and FIT is True:
    pred = train_predict(inputs, outputs, LinearRegression, model_params,
                         int_fn=lin_integers_fn, name=name, filename=None)


### Inputs: all

In [ ]:
model_params = {"l1": 0.001, "max_iter": 5000}

#inputs_tab = {"matrix": matrix_inputs, "num_cp": scalar_inputs}
#inputs_tab = {}
#inputs_tab = {}

if LINEAR is True and FIT is True:

    for name, inputs_list in inputs_tab.items():
        inputs = DataStructure(inputs_list, infer=cydata_all)
        pred = train_predict(inputs, outputs, LinearRegression, model_params,
                             int_fn=lin_integers_fn, name=name,
                             filename=outliers_name("linreg_summary_nonopt_" + name))

In [ ]:
inputs_list = all_inputs
inputs = DataStructure(inputs_list, infer=cydata_all)
name = "all"

model_params = {"l1": 0.1, "l2": 0.001, "max_iter": 5000, "fit_intercept": False}

if LINEAR is True and FIT is True:
    pred = train_predict(inputs, outputs, LinearRegression, model_params,
                         int_fn=lin_integers_fn, name=name,
                         filename=outliers_name("linreg_summary_" + name))

if LINEAR is True and LC is True:
    model_fn = lambda: LinearRegression(inputs, outputs,
                                        model_params=model_params, n=1)

    acc, fig = learning_curve(model_fn, lin_integers_fn,
                              filename=outliers_name("linreg_learning_curve_" + name))

    display(fig)
    print(acc)

## SVM

### Input: matrix

In [ ]:
inputs_list = ['matrix']
inputs = DataStructure(inputs_list)
name = 'matrix'

model_params = {"kernel": "rbf", "C": 20, "gamma": "scale", "epsilon": 0.1,
                "max_iter": -1, "verbose": 0}

if NONLINEAR is True and FIT is True:
    pred = train_predict(inputs, outputs, SVM, model_params,
                         int_fn=integers_fn, name=name,
                         filename=outliers_name("svm_summary_" + name))

if NONLINEAR is True and LC is True:
    model_fn = lambda: SVM(inputs, outputs, model_params=model_params, n=1)

    acc, fig = learning_curve(model_fn, integers_fn,
                              filename=outliers_name("svm_learning_curve_" + name))

    display(fig)
    print(acc)


### Input: num_cp

In [ ]:
inputs_list = ['num_cp']
inputs = DataStructure(inputs_list)
name = 'num_cp'

model_params = {"kernel": "rbf", "C": 20, "gamma": "scale", "epsilon": 0.1,
                "max_iter": -1, "verbose": 0}

if NONLINEAR is True and FIT is True:
    pred = train_predict(inputs, outputs, SVM, model_params,
                         int_fn=integers_fn, name=name,
                         filename=outliers_name("svm_summary_" + name))

if NONLINEAR is True and LC is True:
    model_fn = lambda: SVM(inputs, outputs, model_params=model_params, n=1)

    acc, fig = learning_curve(model_fn, integers_fn,
                                filename=outliers_name("svm_learning_curve_" + name))

    display(fig)
    print(acc)


### Input: num_cp + matrix

In [ ]:
inputs_list = ['num_cp', 'matrix']
inputs = DataStructure(inputs_list)
name = 'num_cp_matrix'

model_params = {"kernel": "rbf", "C": 20, "gamma": "scale", "epsilon": 0.1,
                "max_iter": -1, "verbose": 0}

if NONLINEAR is True and FIT is True:
    pred = train_predict(inputs, outputs, SVM, model_params,
                         int_fn=integers_fn, name=name,
                         filename=outliers_name("svm_summary_" + name))

if NONLINEAR is True and LC is True:
    model_fn = lambda: SVM(inputs, outputs, model_params=model_params, n=1)

    acc, fig = learning_curve(model_fn, integers_fn,
                                filename=outliers_name("svm_learning_curve_" + name))

    display(fig)
    print(acc)


### Input: vectors

In [ ]:
inputs_list = ['dim_cp', 'dim_h0_amb']
inputs = DataStructure(inputs_list)
name = 'vectors'

model_params = {"kernel": "rbf", "C": 10, "gamma": "scale", "epsilon": 0.1,
                "max_iter": -1, "verbose": 0}

if NONLINEAR is True and FIT is True:
    pred = train_predict(inputs, outputs, SVM, model_params,
                         int_fn=integers_fn, name=name,
                         filename=outliers_name("svm_summary_" + name))

# if NONLINEAR is True and LC is True:
#     model_fn = lambda: SVM(inputs, outputs, model_params=model_params, n=1)

#     acc, fig = learning_curve(model_fn, integers_fn,
#                               filename=outliers_name("svm_learning_curve_" + name))

#     display(fig)
#     # print(json.dumps(acc, indent=2))
#     print(acc)


### Input: all

In [ ]:
model_params = {"kernel": "rbf", "C": 30, "gamma": 0.03, "epsilon": 0.1,
                "max_iter": -1, "verbose": 0}

#inputs_tab = {"matrix": matrix_inputs, "num_cp": scalar_inputs}
#inputs_tab = {}
#inputs_tab = {}

if NONLINEAR is True and FIT is True:

    for name, inputs_list in inputs_tab.items():
        inputs = DataStructure(inputs_list, infer=cydata_all)
        pred = train_predict(inputs, outputs, SVM, model_params,
                             int_fn=integers_fn, name=name,
                             filename=outliers_name("svm_summary_nonopt_" + name))

In [ ]:
inputs_list = all_inputs
# inputs_list = ['matrix', 'num_cp', 'norm_matrix', 'dim_cp', 'dim_h0_amb']
inputs = DataStructure(inputs_list)
name = 'all'

model_params = {"kernel": "rbf", "C": 1, "gamma": "scale", "epsilon": 0.1,
                "max_iter": -1, "verbose": 0}

if NONLINEAR is True and FIT is True:
    pred = train_predict(inputs, outputs, SVM, model_params,
                         int_fn=integers_fn, name=name,
                         filename=outliers_name("svm_summary_" + name))

if NONLINEAR is True and LC is True:
    model_fn = lambda: SVM(inputs, outputs, model_params=model_params, n=1)

    acc, fig = learning_curve(model_fn, integers_fn,
                              filename=outliers_name("svm_learning_curve_" + name))

    display(fig)
    print(acc)


## Decision trees

### Input: matrix

In [ ]:
inputs_list = ['matrix']
inputs = DataStructure(inputs_list)
name = 'matrix'

model_params = {"max_depth": None, "min_samples_split": 2,
                "min_samples_leaf": 1, "max_leaf_nodes": None}

# if NONLINEAR is True and FIT is True:
#     pred = train_predict(inputs, outputs, DecistionTree, model_params,
#                          int_fn=integers_fn, name=name,
#                          filename=outliers_name("tree_summary_" + name))

# if LINEAR is True and LC is True:
#     model_fn = lambda: LinearRegression(inputs, outputs,
#                                         model_params=model_params, n=1)

#     acc, fig = learning_curve(model_fn, lin_integers_fn,
#                               filename=outliers_name("linreg_learning_curve_" + name))

#     display(fig)
#     # print(json.dumps(acc, indent=2))
#     print(acc)


### Inputs: all

In [ ]:
model_params = {"max_depth": None, "min_samples_split": 2,
                "min_samples_leaf": 1, "max_leaf_nodes": None}

#inputs_tab = {"matrix": matrix_inputs, "num_cp": scalar_inputs}
#inputs_tab = {}
#inputs_tab = {}

# if NONLINEAR is True and FIT is True:

#     for name, inputs_list in inputs_tab.items():
#         inputs = DataStructure(inputs_list, infer=cydata_all)
#         pred = train_predict(inputs, outputs, DecisionTree, model_params,
#                              int_fn=integers_fn, name=name,
#                              filename=outliers_name("tree_summary_nonopt_" + name))

In [ ]:
inputs_list = all_inputs
inputs = DataStructure(inputs_list, infer=cydata_all)
name = "all"

model_params = {"max_depth": None, "min_samples_split": 2,
                "min_samples_leaf": 1, "max_leaf_nodes": None}

# if NONLINEAR is True and FIT is True:
#     pred = train_predict(inputs, outputs, DecistionTree, model_params,
#                          int_fn=integers_fn, name=name,
#                          filename=outliers_name("tree_summary_" + name))

# if NONLINEAR is True and LC is True:
#     model_fn = lambda: DecistionTree(inputs, outputs,
#                                      model_params=model_params, n=1)

#     acc, fig = learning_curve(model_fn, integers_fn,
#                               filename=outliers_name("tree_learning_curve_" + name))

#     display(fig)
#     print(acc)

## Random forests

### Input: matrix

In [ ]:
inputs_list = ['matrix']
inputs = DataStructure(inputs_list)
name = 'matrix'

model_params = {"n_estimators": 300, "max_depth": None,
                "min_samples_split": 2, "min_samples_leaf": 1,
                "max_leaf_nodes": None, "max_samples": None}

if NONLINEAR is True and FIT is True:
    pred = train_predict(inputs, outputs, RandomForest, model_params,
                         int_fn=integers_fn, name=name,
                         filename=outliers_name("forest_summary_" + name))

if NONLINEAR is True and LC is True:
    model_fn = lambda: RandomForest(inputs, outputs,
                                        model_params=model_params, n=1)

    acc, fig = learning_curve(model_fn, lin_integers_fn,
                              filename=outliers_name("forest_learning_curve_" + name))

    display(fig)
    # print(json.dumps(acc, indent=2))
    print(acc)


### Inputs: all

In [ ]:
model_params = {"n_estimators": 300, "max_depth": None,
                "min_samples_split": 2, "min_samples_leaf": 1,
                "max_leaf_nodes": None, "max_samples": None}

#inputs_tab = {"matrix": matrix_inputs, "num_cp": scalar_inputs}
#inputs_tab = {}
#inputs_tab = {}

if NONLINEAR is True and FIT is True:

    for name, inputs_list in inputs_tab.items():
        inputs = DataStructure(inputs_list, infer=cydata_all)
        pred = train_predict(inputs, outputs, RandomForest, model_params,
                             int_fn=integers_fn, name=name,
                             filename=outliers_name("forest_summary_nonopt_" + name))

In [ ]:
inputs_list = all_inputs
inputs = DataStructure(inputs_list, infer=cydata_all)
name = "all"

model_params = {"n_estimators": 100, "max_depth": None,
                "min_samples_split": 2, "min_samples_leaf": 1,
                "max_leaf_nodes": None, "max_samples": None}

if NONLINEAR is True and FIT is True:
    pred = train_predict(inputs, outputs, RandomForest, model_params,
                         int_fn=lin_integers_fn, name=name,
                         filename=outliers_name("forest_summary_" + name))

if NONLINEAR is True and LC is True:
    model_fn = lambda: RandomForest(inputs, outputs,
                                    model_params=model_params, n=1)

    acc, fig = learning_curve(model_fn, lin_integers_fn,
                              filename=outliers_name("forest_learning_curve_" + name))

    display(fig)
    print(acc)

## Neural networks

### Convolutional model

In [ ]:
def simple_model(inputs, outputs=None,
                 lr=0.001, l1=0., l2=0., momentum=0.9,
                 Ndense=4, Nconv=4,
                 gen_dropout_conv=0., gen_dropout_dense=0.,
                 dropout_conv=0., dropout_dense=0.,
                 alpha=0.1,
                 kernel_size=(4, 4), pool_size=(4, 4),
                 optimizer='adam', loss='mse', rho=0.5):

    # rho: h11 loss weight

    if isinstance(Ndense, int):
        Ndense = [Ndense]
    if isinstance(Nconv, int):
        Nconv = [Nconv]

    if optimizer == 'rmsprop':
        optimizer = keras.optimizers.RMSprop(lr=lr)
    else:
        optimizer = keras.optimizers.Adam(lr=lr)

    l1l2_reg = keras.regularizers.l1_l2(l1=l1, l2=l2)

    input_layers = {}
    output_layers = {}

    # tf.keras.backend.cast(input, dtype='float64')

    matrix_input = layers.Input(shape=inputs.shapes["matrix"],
                                name="matrix_input")

    input_layers["matrix"] = matrix_input

    x = matrix_input

    for i, units in enumerate(Nconv):
        x = layers.Conv2D(units, kernel_size, padding='same',
                          kernel_regularizer=l1l2_reg)(x)
        x = layers.BatchNormalization(momentum=momentum)(x)
        x = layers.LeakyReLU(alpha=alpha)(x)
        x = layers.Dropout(dropout_conv)(x)

    x = layers.Flatten()(x)
    # x = layers.GlobalMaxPooling2D()(x)

    if gen_dropout_conv > 0:
        x = layers.Dropout(gen_dropout_conv)(x)

    conv_output = x

    # general dense layers

    for units in Ndense:
        x = layers.Dense(units, kernel_regularizer=l1l2_reg)(x)
        x = layers.BatchNormalization(momentum=momentum)(x)
        x = layers.LeakyReLU(alpha=alpha)(x)

    if (gen_dropout_conv == 0 or len(Ndense) > 0) and gen_dropout_dense > 0:
        x = layers.Dropout(gen_dropout_dense)(x)

    h11_output = layers.Dense(1, name="h11_output")(x)
    h21_output = layers.Dense(1, name="h21_output")(x)

    if "h11" in outputs.features:
        output_layers["h11"] = h11_output
    if "h21" in outputs.features:
        output_layers["h21"] = h21_output

    if "h11" in outputs.features and "h21" in outputs.features:
        loss_weights = {"h11_output": rho, "h21_output": 1 - rho}
    else:
        loss_weights = None

    model = keras.models.Model(inputs=input_layers, outputs=output_layers)

    model.compile(optimizer=optimizer, loss=loss, loss_weights=loss_weights,
                  metrics=['mse', 'mae'])

    return {"model": model}

In [ ]:
simple_nn_inputs = DataStructure(matrix_inputs, infer=cydata_all)

In [ ]:
%%script false

bhjm_model_params = {"optimizer": "adam",
                     "loss": "mse",
                     "lr": 0.001,
                     "l2": 0.0001,
                     "momentum": 0.99,
                     "alpha": 0.,
                     "dropout_dense": 0.2,
                     "Nconv": [],
                     "Ndense": [875, 460, 440, 930],
                     "rho": 0.5
                     }
bhjm_train_params = {"epochs": 500,
                     "batch_size": 64,
                     "verbose": 0,
                     "early_stopping": (1e-4, 50)
                     }

bhjm_nn = NeuralNet(simple_model, simple_nn_inputs, h11_outputs,
                    model_params=bhjm_model_params, n=5)
bhjm_nn.fit(cydata, train_params=bhjm_train_params)
bhjm_nn_pred = build_pred(bhjm_nn, integers_fn)
print_results(bhjm_nn_pred)
bhjm_nn_pred.summary(filename="bhjm_nn", density=False, logtime=True,
                      training_metrics=['loss']);


In [ ]:
conv_model_params = {"optimizer": "adam",
                     "loss": "mse",
                     "lr": 0.001,
                     "l1": 1e-5,
                     "l2": 1e-5,
                     "momentum": 0.99,
                     "alpha": 0.,
                     "gen_dropout_conv": 0.2,
                     "dropout_conv": 0.,
                     "dropout_dense": 0.,
                     "kernel_size": (3, 3),
                    #  "Nconv": [250, 150, 100, 20],
                     "Nconv": [180, 100, 40, 20],
                     "Ndense": [],
                     "rho": 0.5
                     }
conv_train_params = {"epochs": 2000,
                     "batch_size": 32,
                     "verbose": 0,
                     "early_stopping": (1e-4, 150),
                     "reduce_lr": (0.3, 75)
                     }

if CONVNET is True and FIT is True:

    conv_nn = NeuralNet(simple_model, simple_nn_inputs, h11_outputs,
                        model_params=conv_model_params, n=1)
    conv_nn.fit(cydata, train_params=conv_train_params)
    conv_nn_pred = build_pred(conv_nn, integers_fn)
    print_results(conv_nn_pred)
    conv_nn_pred.summary(filename=outliers_name("conv_nn_summary"), density=False, logtime=True,
                        training_metrics=['loss']);

if CONVNET is True and LC is True:
    ratios = [0.3, 0.8]
    model_fn = lambda: NeuralNet(simple_model, simple_nn_inputs, h11_outputs,
                                 model_params=conv_model_params, n=3)
    fig, acc = learning_curve(model_fn, integers_fn, ratios=ratios,
                              filename=outliers_name("conv_nn_learning_curve"),
                              train_params=conv_train_params)
    fig


### Inception model

In [ ]:
def inception_model(inputs, outputs=None,
                    lr=0.001, l1=0., l2=0., momentum=0.9,
                    Ndense=32, Nconv=32,
                    dropout_conv=0., dropout_dense=0.,
                    alpha=0.1,
                    optimizer='adam', loss='mse', rho=0.5):

    if isinstance(Ndense, int):
        Ndense = [Ndense]
    if isinstance(Nconv, int):
        Nconv = [Nconv]

    if optimizer == 'rmsprop':
        optimizer = keras.optimizers.RMSprop(lr=lr)
    else:
        optimizer = keras.optimizers.Adam(lr=lr)

    l1l2_reg = keras.regularizers.l1_l2(l1=l1, l2=l2)

    input_layers = {}
    output_layers = {}

    matrix_input = layers.Input(shape=inputs.shapes["matrix"],
                                name="matrix_input")

    input_layers["matrix"] = matrix_input

    x = matrix_input

    for i, units in enumerate(Nconv):
        kernel1 = (1, inputs.shapes['matrix'][1])
        # kernel1 = (3, 3)

        x1 = layers.Conv2D(units, kernel1,
                           padding='same', kernel_regularizer=l1l2_reg)(x)
        x1 = layers.LeakyReLU(alpha=alpha)(x1)

        kernel2 = (inputs.shapes['matrix'][0], 1)
        # kernel2 = (5, 5)
        x2 = layers.Conv2D(units, kernel2,
                           padding='same', kernel_regularizer=l1l2_reg)(x)
        x2 = layers.LeakyReLU(alpha=alpha)(x2)

        x = layers.concatenate([x1, x2])
        x = layers.BatchNormalization(momentum=momentum)(x)

    if dropout_conv > 0:
        x = layers.Dropout(dropout_conv)(x)

    x = layers.Flatten()(x)

    # general dense layers

    for units in Ndense:
        x = layers.Dense(units, kernel_regularizer=l1l2_reg)(x)
        x = layers.LeakyReLU(alpha=alpha)(x)
        x = layers.BatchNormalization(momentum=momentum)(x)

    if len(Ndense) > 0 and dropout_dense > 0:
        x = layers.Dropout(dropout_dense)(x)

    h11_output = layers.Dense(1, activation="relu", name="h11_output")(x)
    h21_output = layers.Dense(1, activation="relu", name="h21_output")(x)

    if "h11" in outputs.features:
        output_layers["h11"] = h11_output
    if "h21" in outputs.features:
        output_layers["h21"] = h21_output

    if "h11" in outputs.features and "h21" in outputs.features:
        loss_weights = {"h11_output": rho, "h21_output": 1 - rho}
    else:
        loss_weights = None

    model = keras.models.Model(inputs=input_layers, outputs=output_layers)

    model.compile(optimizer=optimizer, loss=loss, loss_weights=loss_weights,
                  metrics=['mse', 'mae'])

    return {"model": model}

In [ ]:
inc_outputs = h11_outputs
# inc_outputs = outputs

inc_model_params = {"optimizer": "adam",
                    "loss": "mse",
                    "lr": 0.001,
                    "l1": 1e-4,
                    "l2": 1e-4,
                    "momentum": 0.99,
                    "alpha": 0.,
                    "dropout_conv": 0.2,
                    "dropout_dense": 0.,
                    "Nconv": [32, 64, 32],
                    "Ndense": [],
                    "rho": 0.5
                    }
inc_train_params = {"epochs": 2000,
                    "batch_size": 32,
                    "verbose": 0,
                    "early_stopping": (1e-4, 200),
                    "reduce_lr": (0.3, 75)
                    }

if INCNET is True and FIT is True:
    inc_nn = NeuralNet(inception_model, simple_nn_inputs, inc_outputs,
                        model_params=inc_model_params, n=3)
    inc_nn.fit(cydata, train_params=inc_train_params)
    inc_nn_pred = build_pred(inc_nn, integers_fn)
    print_results(inc_nn_pred)
    inc_nn_pred.summary(filename=outliers_name("inc_nn_summary"), density=False,
                        logtime=True, training_metrics=['loss']);
    
    inc_nn_pred.training_curve("mae", filename="h11_training_curve_mae.pdf", logtime=True)
    inc_nn_pred.training_curve("mse", filename="h11_training_curve_mse.pdf", logtime=True)
    inc_nn_pred.plot_feature("h11", plottype="plain", sigma=0, filename="h11_dist_pred_plain.pdf", logtime=True)
    inc_nn_pred.plot_feature("h11", plottype="step", sigma=0, filename="h11_dist_pred_step.pdf", logtime=True)

if INCNET is True and LC is True:

    ratios = [0.3, 0.8]
    # ratios = None
    suffix = "_1d_kernel_out"

    model_fn = lambda: NeuralNet(inception_model, simple_nn_inputs, inc_outputs,
                                 model_params=inc_model_params, n=2)
    fig, acc = learning_curve(model_fn, integers_fn, ratios=ratios,
                              filename=outliers_name("inc_nn_learning_curve" + suffix),
                              train_params=inc_train_params)
    fig


In [ ]:
%%script false

lc_outputs = h11_outputs
# lc_outputs = outputs

inc_model_params = {"optimizer": "adam",
                    "loss": "mse",
                    "lr": 0.001,
                    "l1": 1e-4,
                    "l2": 1e-3,
                    "momentum": 0.99,
                    "alpha": 0.,
                    "dropout_conv": 0.2,
                    "dropout_dense": 0.,
                    "Nconv": [32, 64, 32],
                    "Ndense": [],
                    "rho": 0.5
                    }
inc_train_params = {"epochs": 2000,
                    "batch_size": 32,
                    "verbose": 0,
                    "early_stopping": (1e-4, 200),
                    "reduce_lr": (0.3, 75)
                    }

ratios = np.arange(0.1, 1, 0.1)
model_fn = lambda: NeuralNet(inception_model, simple_nn_inputs, h11_outputs,
                             model_params=inc_model_params, n=5)
fig, acc = learning_curve(model_fn, integers_fn, ratios=ratios,
                          filename=outliers_name("inc_nn_learning_curve_stat"),
                          train_params=inc_train_params)
fig

### Advanced model

In [ ]:
def advanced_model(inputs, outputs=None,
                   lr=0.001, l1=0., l2=0., momentum=0.9,
                   Ndense=32, Nconv=32,
                   dropout_conv=0., dropout_dense=0.,
                   alpha=0.1,
                   optimizer='adam', loss='mse', rho=0.5):

    if isinstance(Ndense, int):
        Ndense = [Ndense]
    if isinstance(Nconv, int):
        Nconv = [Nconv]

    if optimizer == 'rmsprop':
        optimizer = keras.optimizers.RMSprop(lr=lr)
    else:
        optimizer = keras.optimizers.Adam(lr=lr)

    l1l2_reg = keras.regularizers.l1_l2(l1=l1, l2=l2)

    input_layers = {}
    output_layers = {}

    matrix_input = layers.Input(shape=inputs.shapes["matrix"],
                                name="matrix_input")

    num_cp_input = layers.Input(shape=(1,), name="num_cp_input")
    num_eqs_input = layers.Input(shape=(1,), name="num_eqs_input")
    norm_matrix_input = layers.Input(shape=(1,), name="norm_matrix_input")

    dim_cp_input = layers.Input(shape=inputs.shapes["dim_cp"],
                                name="dim_cp_input")
    dim_h0_amb_input = layers.Input(shape=inputs.shapes["dim_h0_amb"],
                                    name="dim_h0_amb_input")

    input_layers["matrix"] = matrix_input
    input_layers["num_cp"] = num_cp_input
    input_layers["num_eqs"] = num_eqs_input
    input_layers["norm_matrix"] = norm_matrix_input
    input_layers["dim_cp"] = dim_cp_input
    input_layers["dim_h0_amb"] = dim_h0_amb_input

    # matrix process

    x = matrix_input

    for i, units in enumerate(Nconv):
        x1 = layers.Conv2D(units, (1, 15), padding='same',
                           kernel_regularizer=l1l2_reg)(x)
        x1 = layers.LeakyReLU(alpha=alpha)(x1)

        x2 = layers.Conv2D(units, (12, 1), padding='same',
                           kernel_regularizer=l1l2_reg)(x)
        x2 = layers.LeakyReLU(alpha=alpha)(x2)

        x = layers.concatenate([x1, x2])
        x = layers.BatchNormalization(momentum=momentum)(x)

    if dropout_conv > 0:
        x = layers.Dropout(dropout_conv)(x)

    matrix_result = layers.Flatten()(x)

    # scalars and dim_cp process

    x = layers.concatenate([num_cp_input, num_eqs_input,
                            # norm_matrix_input,
                            # layers.Flatten()(dim_h0_amb_input),
                            layers.Flatten()(dim_cp_input)
                            ])

    for units in Ndense:
        x = layers.Dense(units, kernel_regularizer=l1l2_reg)(x)
        x = layers.LeakyReLU(alpha=alpha)(x)
        x = layers.BatchNormalization(momentum=momentum)(x)

    if len(Ndense) > 0 and dropout_dense > 0:
        x = layers.Dropout(dropout_dense)(x)

    sc_vec_result = x

    # predict h11

    x = layers.concatenate([num_cp_input, matrix_result])

    h11_output = layers.Dense(1, activation="relu", name="h11_output")(x)

    # predict h21

    if "h11" in outputs.features:
        x = layers.concatenate([h11_output, sc_vec_result, matrix_result,
                                layers.Flatten()(dim_h0_amb_input)
                                ])
    else:
        x = layers.concatenate([sc_vec_result, matrix_result,
                                layers.Flatten()(dim_h0_amb_input),
                                ])

    h21_output = layers.Dense(1, activation="relu", name="h21_output")(x)

    if "h11" in outputs.features:
        output_layers["h11"] = h11_output
    if "h21" in outputs.features:
        output_layers["h21"] = h21_output

    if "h11" in outputs.features and "h21" in outputs.features:
        loss_weights = {"h11_output": rho, "h21_output": 1 - rho}
    else:
        loss_weights = None

    model = keras.models.Model(inputs=input_layers, outputs=output_layers)

    model.compile(optimizer=optimizer, loss=loss, loss_weights=loss_weights,
                  metrics=['mse', 'mae'])

    return {"model": model}


In [ ]:
adv_model_params = {"optimizer": "adam",
                    "loss": "mse",
                    "lr": 0.001,
                    "l1": 1e-4,
                    "l2": 1e-4,
                    "momentum": 0.9,
                    "alpha": 0.1,
                    "dropout_conv": 0.2,
                    "dropout_dense": 0.2,
                    "Nconv": [32, 32],
                    "Ndense": [30],
                    "rho": 0.4
                    }
adv_train_params = {"epochs": 2000,
                    "batch_size": 32,
                    "verbose": 0,
                    "early_stopping": (1e-4, 150),
                    "reduce_lr": (0.3, 50)
                    }

if ADVNET is True and FIT is True:

    adv_nn_inputs_list = ["matrix", "num_cp", "num_eqs", "norm_matrix",
                        "dim_cp", "dim_h0_amb"]
    adv_nn_inputs = DataStructure(adv_nn_inputs_list, infer=cydata_all)

    adv_nn = NeuralNet(advanced_model, adv_nn_inputs, outputs,
                    model_params=adv_model_params, n=1)
    adv_nn.fit(cydata, train_params=adv_train_params)
    adv_nn_pred = build_pred(adv_nn, integers_fn)
    print_results(adv_nn_pred)
    adv_nn_pred.summary(filename=outliers_name("adv_nn_summary"), density=False,
                        logtime=True, training_metrics=['loss']);

ratios = np.arange(0.2, 0.9, 0.3)
# ratios = np.arange(0.2, 0.9, 0.2)

if ADVNET is True and LC is True:
    model_fn = lambda: NeuralNet(inception_model, simple_nn_inputs, h21_outputs,
                                 model_params=adv_model_params, n=1)
    fig, acc = learning_curve(model_fn, integers_fn, ratios=ratios,
                              filename=outliers_name("adv_nn_learning_curve"),
                              train_params=adv_train_params)
    fig
